# Practice training a deep neural network on the CIFAR10 image dataset:

In [ ]:
import os
os.chdir("..")

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms
import torch.optim as optim

from PIL import Image
import time

import numpy as np
from pathlib import Path

a. Build a DNN with 20 hidden layers of 100 neurons each (that’s too many, but it’s the point of this exercise). Use He initialization and the Swish activation function.

In [267]:
batch_size = 64

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

# Load CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset  = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Split training data into train and validation subsets (e.g., 80%/20%)
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_subset, val_subset = torch.utils.data.random_split(train_dataset, [train_size, val_size])

train_loader = torch.utils.data.DataLoader(train_subset, batch_size=batch_size, shuffle=True)
val_loader   = torch.utils.data.DataLoader(val_subset, batch_size=batch_size, shuffle=False)
test_loader  = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [286]:
def weights_init(layer):
    if isinstance(layer, nn.Linear):
        torch.nn.init.kaiming_normal_(layer.weight, mode='fan_in', nonlinearity='leaky_relu')
        torch.nn.init.constant_(layer.bias, 0)

class Swish(nn.Module):
    def forward(self, x):
        return x * F.sigmoid(x)

In [287]:
class CIFAR10(nn.Module):
    def __init__(self, input_features=3*32*32, output_neurons=100, num_classes=10, hidden_layers=20):
        super(CIFAR10, self).__init__()
        layers = []
        
        layer_first = nn.Linear(input_features, output_neurons)
        layers.append(layer_first)
        layers.append(Swish())
        
        for i in range(hidden_layers-1):
            layer = nn.Linear(output_neurons, output_neurons)
            layers.append(layer)
            layers.append(Swish())

        layer_last = nn.Linear(output_neurons, num_classes)
        layers.append(layer_last)
        
        self.net = nn.Sequential(*layers)
        self.net.apply(weights_init)

    def forward(self, X):
        flatten = nn.Flatten()
        X = flatten(X)
        return self.net(X)

In [293]:
def train(model, train_loader, val_loader, epochs, criterion, optimizer, patience=5, device='cpu'):
    best_val_loss = 0.0
    patience_counter = 0
    
    for epoch in range(epochs):
        model.train()
        per_epoch_train_loss = 0.0
        for data in train_loader:
            inputs, targets = data[0].to(device), data[1].to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            per_epoch_train_loss += loss.item()

        avg_per_epoch_train_loss = per_epoch_train_loss / len(train_loader)

        per_epoch_val_loss = 0.0
        total = 0
        correct = 0
        for data in val_loader:
            inputs, targets = data[0].to(device), data[1].to(device)
            
            with torch.no_grad():
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                total += targets.size(0)
                correct += (predicted == targets).sum().item()
                
                loss = criterion(outputs, targets)
                per_epoch_val_loss += loss.item()
                
        avg_per_epoch_val_loss = per_epoch_val_loss / len(val_loader)
        val_accuracy = 100* correct / total

        print(f"Epoch [{epoch+1}/{epochs}], Train Loss: {avg_per_epoch_train_loss:.4f}, " 
              f"Val Loss: {avg_per_epoch_val_loss:.4f}, Val Acc: {val_accuracy:.2f}%")

        if best_val_loss <= avg_per_epoch_val_loss:
            patience_counter = 0
            best_val_loss = avg_per_epoch_val_loss
            best_model_state = model.state_dict()
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping triggered.")
                model.load_state_dict(best_model_state)
                break

    return model

In [294]:
cifar10  = CIFAR10()
criterion = nn.CrossEntropyLoss()
optimizer = optim.NAdam(cifar10.parameters(), lr=0.001)
model = train(cifar10, train_loader, val_loader, 5, criterion, optimizer)

Epoch [1/5], Train Loss: 1.8841, Val Loss: 1.7397, Val Acc: 36.88%
Epoch [2/5], Train Loss: 1.6469, Val Loss: 1.6521, Val Acc: 41.69%
Epoch [3/5], Train Loss: 1.5441, Val Loss: 1.6020, Val Acc: 43.94%
Epoch [4/5], Train Loss: 1.4674, Val Loss: 1.5347, Val Acc: 45.58%
Epoch [5/5], Train Loss: 1.4014, Val Loss: 1.5526, Val Acc: 46.05%


CIFAR10(
  (net): Sequential(
    (0): Linear(in_features=3072, out_features=100, bias=True)
    (1): Swish()
    (2): Linear(in_features=100, out_features=100, bias=True)
    (3): Swish()
    (4): Linear(in_features=100, out_features=100, bias=True)
    (5): Swish()
    (6): Linear(in_features=100, out_features=100, bias=True)
    (7): Swish()
    (8): Linear(in_features=100, out_features=100, bias=True)
    (9): Swish()
    (10): Linear(in_features=100, out_features=100, bias=True)
    (11): Swish()
    (12): Linear(in_features=100, out_features=100, bias=True)
    (13): Swish()
    (14): Linear(in_features=100, out_features=100, bias=True)
    (15): Swish()
    (16): Linear(in_features=100, out_features=100, bias=True)
    (17): Swish()
    (18): Linear(in_features=100, out_features=100, bias=True)
    (19): Swish()
    (20): Linear(in_features=100, out_features=100, bias=True)
    (21): Swish()
    (22): Linear(in_features=100, out_features=100, bias=True)
    (23): Swish()
    (24